## Wstęp do Uczenia Maszynowego

#### Laboratorium 01

### 0\. Sprawy organizacyjne

Kontakt: MS Teams, anna.kozak@pw.edu.pl, katarzyna.woznica@pw.edu.pl

Pracujemy z językiem `Python`.

Pracujemy z repozytorium GitHub: [https://github.com/kozaka93/2025Z-MachineLearning](https://github.com/kozaka93/2025Z-MachineLearning)

Na repozytorium będą się pojawiały pliki `.ipynb` z materiałami na zajęcia.

Rozwiązania prac domowych i projektu będę przesyłane na wskazany adres mail w wyznaczonym terminie.

*Zasady zaliczenia:*

  - 3 x praca domowa (3 x 15p) - raport + kod
  - projekt (35p) - raport + kod + odpowiedź ustna

Aby zaliczyć laboratoria, należy uzyskać ponad 40 punktów ogółem, w tym co najmniej 15 punktów z projektu. Na podstawie punktów jest wystawiana ocena z części laboratoryjnej.

| Ocena | 3 | 3.5 | 4 | 4.5 | 5 |
|:---:|:---:|:---:|:---:|:---:|:---:|
| Punkty | (40, 48] | (48, 56] | (56, 64] | (64, 72] | (72, ∞) |

Z oceną za laboratoria przystępuje się do egzaminu ustnego.

-----

### 1\. Czym będziemy się zajmować?



<img src="tematy.png" alt="drawing" width="800">

W uczeniu maszynowym pracujemy ze zbiorem danych $\mathbf{D}= (X, Y)$.

  - $X$ to **macierz cech** (zmiennych niezależnych) o wymiarze $n \times p$ ($n$ próbek, $p$ cech).
  - $Y$ to **wektor zmiennej docelowej** (etykieta, wynik, który chcemy przewidzieć) o wymiarze $n$.

#### Rodzaje Uczenia Maszynowego

1.  **Uczenie nadzorowane (Supervised Learning):** Jeżeli wartość $Y$ jest znana w zbiorze treningowym (czyli mamy dane wejściowe **i** poprawne odpowiedzi). Chcemy, aby model nauczył się mapowania $X \rightarrow Y$.
      * **Regresja (Regression):** Gdy zmienna $Y$ jest ciągła (np. zarobki, temperatura, cena domu, czyli $Y \in R$).
      * **Klasyfikacja (Classification):** Gdy zmienna $Y$ jest dyskretna (kategorie, klasy, np. tak/nie, kot/pies/mysz, czyli $Y \in \{0, 1, 2, ..\}$). Gdy $Y \in \{0,1\}$ mówimy o klasyfikacji binarnej.
2.  **Uczenie nienadzorowane (Unsupervised Learning):** Jeżeli nie znamy wartości $Y$ (brakuje nam etykiet). Celem jest znalezienie ukrytych wzorców i struktury w samych danych $X$ (np. grupowanie danych w skupienia).

-----

### 2\. Jak wygląda proces uczenia?

Proces uczenia maszynowego polega na stworzeniu **modelu** $\mathbf{M}$, który na podstawie danych wejściowych $X$ potrafi przewidzieć wartość docelową $\hat{Y}$.

**Model:** $\mathbf{M}: D \rightarrow \hat{Y}$

**Prognoza:** $\hat{Y} = M(X)$ .

Model jest "uczony" na danych $D$, aby jak najlepiej minimalizować błąd między przewidywanym $\hat{Y}$ a rzeczywistym $Y$.

-----

### 3\. Etapy uczenia maszynowego

#### 3.1 Pakiety i dane



In [ ]:
# Import potrzebnych pakietów (bibliotek)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 'sklearn' (scikit-learn)  biblioteka do uczenia maszynowego w Pythonie.
# Z niej importujemy konkretne algorytmy, np. 'tree' (Drzewa Decyzyjne).
from sklearn import tree

# Będziemy też potrzebować funkcji do podziału danych
from sklearn.model_selection import train_test_split

# Metryka do oceny modelu (np. błęd średniokwadratowy)
from sklearn.metrics import mean_squared_error

Rozważmy zbiór danych zawierający informacje o zarobkach zawodników Baseballu.

In [ ]:
# Zbiory danych można wczytać przez link z GitHub, nie ma koniecznośći ich pobierania.
# index_col =[0] mówi, aby pierwszą kolumnę pliku CSV potraktować jako indeks wierszy.
Hitters = pd.read_csv("https://raw.githubusercontent.com/kozaka93/2025Z-MachineLearning/refs/heads/main/labs/lab01/Hitters.csv", index_col =[0])

In [ ]:
Hitters

-----

#### 3.2 Przygotowanie danych (Pre-processing)

##### *Zadanie 1*

-----

Przygotuj zbiór danych do zbudowania pierwszego modelu.

a) Utwórz zbiór *Hitters\_small* zawierający zmienne o nazwach *Years, Hits, Salary*.

b) Przyjrzyj się utworzonemu zbiorowi danych. Jaki jest wymiar danych, typy zmiennych, czy są braki danych? Jakie rozkłady mają zmienne?

c) Podziel zbiór danych na X = ['Years', 'Hits'] (cechy) i y = ['Salary'] (zmienna docelowa).

d) Podziel dane na zbiór treningowy i testowy (proporcja 50:50).

**Rozwiązanie Zadania 1**

In [ ]:
# a) 

# Tworzymy nowy DataFrame, wybierając tylko potrzebne kolumny.
Hitters_small = Hitters[["Years", "Hits", "Salary"]]

Hitters_small

In [ ]:
# b) Analiza danych (Exploratory Data Analysis - EDA)

# 1. Wymiar danych: .shape zwraca krotkę (liczba wierszy, liczba kolumn)
print("Wymiar danych (wiersze x kolumny):", Hitters_small.shape)

# 2. Informacje o kolumnach: .info() pokazuje liczbę wierszy, typy danych i liczbę nie-pustych wartości.
# Patrząc na 'Non-Null Count', widzimy, że kolumna 'Salary' ma braki danych (tylko 263 nie-pustych wartości z 322).
Hitters_small.info()

# 3. Podstawowe statystyki opisowe: .describe() dla zmiennych numerycznych
# Pokazuje średnią (mean), odchylenie standardowe (std), min, max i kwartyle, co daje wgląd w rozkład zmiennych.
Hitters_small.describe()

# 4. Wizualizacja rozkładów: histogramy
# Używamy .hist() z Pandas, aby zobaczyć rozkład każdej zmiennej.
Hitters_small.hist(sharex = False, sharey = False, bins = 50)

In [ ]:
# 5. Model nie może uczyć się na brakujących danych, dlatego musimy je usunąć lub uzupełnić.
# Ponieważ braki są tylko w zmiennej 'Salary' (nasza zmienna docelowa 'y'), najlepiej usunąć całe wiersze.
Hitters_clean = Hitters_small.dropna()

In [ ]:
# c) Podział na cechy (X) i zmienną docelową (y)

# X - Cechy (zmienne niezależne, 'predyktory'): na podstawie Years i Hits będziemy przewidywać Salary.
X = Hitters_clean.drop(["Salary"], axis=1)

# y - Zmienna docelowa (zmienna zależna, 'target'): to, co chcemy przewidzieć.
y = Hitters_clean.Salary

In [ ]:
## Eksperyment - transformacja log zmiennych skośnych

experiment = pd.DataFrame({"Var1" : Hitters.Salary,
                           "Var2": (-Hitters.Salary) + 2000})

experiment["log(Var1)"] = np.log(experiment.Var1)
experiment["log(Var2)"] = np.log(experiment.Var2)

experiment.hist()

In [ ]:
# d) Podział na zbiór treningowy i testowy

# Dzielimy dane na dwie części:
# 1. Zbiór Treningowy (Training Set): Używany do nauki modelu.
# 2. Zbiór Testowy (Test Set): Używany do oceny, jak model radzi sobie z danymi, których 'nie widział'.

# 'train_test_split' z biblioteki sklearn automatycznie dokonuje podziału.
# 'test_size=0.5' oznacza podział 50% na testowy i 50% na treningowy.
# 'random_state=123' to tzw. "ziarno losowości" - użycie stałej liczby sprawia, że podział jest zawsze taki sam. Jest to ważne dla powtarzalności wyników.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

#### 3.3 Podstawy drzew decyzyjnych 

#### *Jak się buduje drzewa dla zadania regresji?*
1. Dzielimy przestrzeń $X = (X_1, X_2, \dots, X_p)$ na $J$ regionów $R_1, R_2, \dots, R_J$.
2. Predykcja w $j$-ty regionie jest równa średniej z wartości obserwacji $y$ zbioru treningowego w regione $R_J$. 

In [ ]:
# Tworzymy obiekt klasy DecisionTreeRegressor.
# Drzewo będzie miało domyślne parametry.
Tree = tree.DecisionTreeRegressor()

# Trening (uczenie) modelu: Używamy metody .fit(), aby nauczyć model na danych treningowych.
Tree.fit(X_train, y_train)

Wizualizacja drzewa decyzyjnego

In [ ]:
tree.plot_tree(Tree,
               feature_names=X.columns.tolist(), # Nazwy kolumn cech
               filled=True # Wypełnienie węzłów kolorem
               ) 
plt.title("Drzewo Decyzyjne")
plt.show()

Przejrzysty zapis otrzymanego drzewa

In [ ]:
from sklearn.tree import export_text
t = export_text(Tree)
print(t)

**Drzewo o trzech liściach**

Poprzez wskazywanie hiperparametrów możemy manipulować skompliowaniem i jakością modelu drzewa. Zbudujmy drzewo o hiperparametrze `max_leaf_nodes` = 3.

In [ ]:
Tree3 = tree.DecisionTreeRegressor(max_leaf_nodes=3)
Tree3 = Tree3.fit(X_train, y_train)

tree.plot_tree(Tree3, 
               feature_names=Tree3.feature_names_in_.tolist(),
               filled=True)

#### 3.4 Jak dobry jest nasz model (zadanie regresji)?

Błąd średniokwadratowy:

$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{f}(x_i))^2$,

gdzie:

$y_i$ - wartość prawdziwa dla $i$-tej obserwacji,

$\hat{f}(x_i)$ jest predykcją modelu dla $i$-tej obserwacji.

In [ ]:
# Generowanie prognoz (Prediction)
# Używamy wytrenowanych modeli do przewidywania wartości 'Salary' na nieznanych danych (X_test).
y_pred = Tree.predict(X_test)
y_pred3 = Tree3.predict(X_test)

# Obliczanie Błędu Średniokwadratowego (Mean Squared Error - MSE)

mse = mean_squared_error(y_test, y_pred)
mse3 = mean_squared_error(y_test, y_pred3)

print(f"MSE dla Tree: {mse:.2f}")
print(f"MSE dla Tree3 (max_depth=3): {mse3:.2f}") 

In [ ]:
# Wizualizacja prognoz na tle rzeczywistych wartości
df = pd.DataFrame({'y_test':y_test, 'y_pred':Tree3.predict(X_test)})

# Tworzymy wykres rozrzutu (scatter plot)
fig, ax = plt.subplots(figsize=(8, 8))

# Dane: rzeczywista pensja (y_test) vs. przewidywana pensja (y_pred)
df.plot(x='y_test', y='y_pred', kind='scatter', ax=ax, title="Prognoza (Y_pred) vs. Rzeczywistość (Y_test) dla Tree3")


# Idealny model miałby wszystkie punkty na linii Y_test = Y_pred
df.plot(x='y_test', y='y_pred', kind='scatter', ax=ax)
ax.plot(np.array([0, 2000]), np.array([0, 2000]), color='red')

# Interpretacja wykresu: Im bliżej czerwonej linii są punkty, tym lepsze są przewidywania modelu.